# TIME experiment

The goal is to try some deep learning on questionnaire + time_series

In [ ]:
import pandas as pd
import numpy as np
from spark.preprocessor import preprocessor

## Loading time series 

In [127]:
raw_data_prep_dir = '../raw_data/pads-parkinsons-disease-smartwatch-dataset-1.0.0/preprocessed/'

In [83]:
file_list = pd.read_csv(raw_data_prep_dir  + "file_list.csv")
file_list

,resource_type,id,study_id,condition,disease_comment,age_at_diagnosis,age,height,weight,gender,handedness,appearance_in_kinship,appearance_in_first_grade_kinship,effect_of_alcohol_on_tremor,label
0,patient,1,PADS,Healthy,-,56,56,173,78,male,right,True,True,Unknown,0
1,patient,2,PADS,Other Movement Disorders,Left-Sided resting tremor and hypokinesia with...,69,81,193,104,male,right,False,NaN,No effect,2
2,patient,3,PADS,Healthy,-,45,45,170,78,female,right,False,NaN,Unknown,0
3,patient,4,PADS,Parkinson's,IPS akinetic-rigid type,63,67,161,90,female,right,False,NaN,No effect,1
4,patient,5,PADS,Parkinson's,IPS tremordominant type,65,75,172,86,male,left,False,NaN,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,patient,465,PADS,Parkinson's,IPS mixed type,62,65,175,80,male,right,True,False,No effect,1
465,patient,466,PADS,Healthy,-,84,84,172,74,female,right,True,True,No effect,0
466,patient,467,PADS,Parkinson's,"Essential Tremor, starting IPS tremordominant ...",55,57,190,100,male,right,False,NaN,Improvement,1
467,patient,468,PADS,Parkinson's,IPS mixed type,73,76,198,118,male,right,False,NaN,No effect,1


In [120]:
n_obs = len(file_list['id'])
n_obs

469

In [128]:
time_data = []
for idx, subject_idx in enumerate(file_list['id']):
    file_name = raw_data_prep_dir + f'movement/{subject_idx:03d}_ml.bin'
    time_idx_data = np.fromfile(file_name, dtype=np.float32).reshape((-1, 976))
    time_data.append(time_idx_data)

In [126]:
len(time_data)

469

In [ ]:
  data = np.fromfile(fileName, dtype=np.float32).reshape((-1, 976))


In [130]:
channels = []
for task in ["Relaxed1", "Relaxed2", "RelaxedTask1", "RelaxedTask2", "StretchHold", "HoldWeight",
                "DrinkGlas", "CrossArms", "TouchNose", "Entrainment1", "Entrainment2"]:
    for device_location in ["LeftWrist", "RightWrist"]:
        for sensor in ["Acceleration", "Rotation"]:
            for axis in ["X", "Y", "Z"]:
                channel = f"{task}_{sensor}_{device_location}_{axis}"
                channels.append(channel)
channels

['Relaxed1_Acceleration_LeftWrist_X',
 'Relaxed1_Acceleration_LeftWrist_Y',
 'Relaxed1_Acceleration_LeftWrist_Z',
 'Relaxed1_Rotation_LeftWrist_X',
 'Relaxed1_Rotation_LeftWrist_Y',
 'Relaxed1_Rotation_LeftWrist_Z',
 'Relaxed1_Acceleration_RightWrist_X',
 'Relaxed1_Acceleration_RightWrist_Y',
 'Relaxed1_Acceleration_RightWrist_Z',
 'Relaxed1_Rotation_RightWrist_X',
 'Relaxed1_Rotation_RightWrist_Y',
 'Relaxed1_Rotation_RightWrist_Z',
 'Relaxed2_Acceleration_LeftWrist_X',
 'Relaxed2_Acceleration_LeftWrist_Y',
 'Relaxed2_Acceleration_LeftWrist_Z',
 'Relaxed2_Rotation_LeftWrist_X',
 'Relaxed2_Rotation_LeftWrist_Y',
 'Relaxed2_Rotation_LeftWrist_Z',
 'Relaxed2_Acceleration_RightWrist_X',
 'Relaxed2_Acceleration_RightWrist_Y',
 'Relaxed2_Acceleration_RightWrist_Z',
 'Relaxed2_Rotation_RightWrist_X',
 'Relaxed2_Rotation_RightWrist_Y',
 'Relaxed2_Rotation_RightWrist_Z',
 'RelaxedTask1_Acceleration_LeftWrist_X',
 'RelaxedTask1_Acceleration_LeftWrist_Y',
 'RelaxedTask1_Acceleration_LeftWrist_Z'

In [132]:
time_data_array = np.array(time_data)
time_data_flat = time_data_array.reshape((n_obs,132*976))
time_data_flat.shape

(469, 128832)